In [1]:
from pygromos.files.topology import top
from pygromos.files.topology import ptp
from pygromos.files.coord import cnf

import copy
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
input_dir = "input"
in_top1_path = input_dir+"/F313.top"
in_top2_path = input_dir+"/G078.top"

in_coord2_path = input_dir+"/G078_unitedatom_optimised_geometry.g96"
in_coord1_path = input_dir+"/F313_unitedatom_optimised_geometry.g96"

out_ptp_path = "./out/fun.ptp"
out_top_path = "./out/fun.top"
out_cnf_path = "./out/coord.g96"


## COORD

In [3]:
cmol1 = cnf.Cnf(in_coord1_path)
cmol2 = cnf.Cnf(in_coord2_path)

In [4]:
cmol2.POSITION

POSITION
# 	 
    1 G078  C12        1   -0.105294327    0.294456128   -0.000958613
    1 G078  C5         2   -0.102027449    0.143424651    0.000075814
    1 G078  C4         3   -0.219248048    0.070784658    0.000720005
    1 G078  H5         4   -0.314580944    0.123211384    0.000514463
    1 G078  C3         5   -0.219247102   -0.070786584    0.000719964
    1 G078  H4         6   -0.314578754   -0.123215832    0.000507374
    1 G078  C2         7   -0.102025935   -0.143424981    0.000073281
    1 G078  C1         8   -0.105292139   -0.294457008   -0.000963503
    1 G078  C11        9    0.022455714   -0.071930088    0.000196196
    1 G078  C6        10    0.022456011    0.071932015    0.000189775
    1 G078  C7        11    0.147542435    0.139715789    0.000206284
    1 G078  H6        12    0.148905210    0.248277889    0.000502444
    1 G078  C8        13    0.266930052    0.070661361   -0.000029547
    1 G078  H7        14    0.361158038    0.124866672    0.000272934
    1 

In [5]:
cmol2.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
 cmol1.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Atom Matching

In [7]:
from rdkit.Chem import rdFMCS

def find_atom_mapping(cnfA:cnf.Cnf, cnfB:cnf.Cnf)->(dict,str):
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)


    #CompareElements
    mcs = rdFMCS.FindMCS([mol1, mol2],
                         atomCompare=rdFMCS.AtomCompare.CompareAnyHeavyAtom,
                         bondCompare=rdFMCS.BondCompare.CompareOrderExact,
                         ringMatchesRingOnly=True, completeRingsOnly=True)
    #                    ringCompare=rdFMCS.RingCompare.StrictRingFusion)
    smartsString = mcs.smartsString

    ##MCS Match
    patt = Chem.MolFromSmarts(smartsString)  # smartsString
    mol1Match = [i+1 for i in mol1.GetSubstructMatch(patt)]
    mol2Match = [i+1 for i in mol2.GetSubstructMatch(patt)]

    atom_mappingAB = dict(zip(mol1Match, mol2Match))
    return atom_mappingAB, smartsString

atom_mappingAB, smart = find_atom_mapping(cnfA=cmol1, cnfB=cmol2)
len(atom_mappingAB), atom_mappingAB, smart

(10,
 {3: 2, 2: 1, 4: 10, 6: 9, 8: 7, 12: 5, 14: 3, 15: 4, 13: 6, 9: 8},
 '[#6&R]1(-&!@[#8,#6;!R])-&@[#6&R]-&@[#6&R]-&@[#6&R](-&@[#6&R](-&@[#6&R]-&@1-&!@[#1&!R])-&!@[#1&!R])-&!@[#7,#6;!R]')

## COORDINATE Generation
### Coord Alignment

In [8]:
def align_cnfs_with_MCS(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atom_mappingAB:int)->(cnf.Cnf, cnf.Cnf):
    
    cnfA= copy.deepcopy(cnfA)
    cnfB= copy.deepcopy(cnfB)
    
    mol1 = Chem.MolFromPDBBlock(cnfA.get_pdb(rdkit_ready=True), removeHs=False)
    mol2 = Chem.MolFromPDBBlock(cnfB.get_pdb(rdkit_ready=True), removeHs=False)
    
    atom_mappingAB = {key-1:val-1 for key, val in atom_mappingAB.items()}
    AllChem.AlignMol(mol1, mol2, atomMap=list(atom_mappingAB.items()))


    conf2 = mol2.GetConformer()
    for ind, pos in enumerate(conf2.GetPositions()):
        cnfB.POSITION[ind].xp= pos[0]/10
        cnfB.POSITION[ind].yp= pos[1]/10
        cnfB.POSITION[ind].zp= pos[2]/10

    conf1 = mol1.GetConformer()
    for ind, pos in enumerate(conf1.GetPositions()):
        cnfA.POSITION[ind].xp= pos[0]/10
        cnfA.POSITION[ind].yp= pos[1]/10
        cnfA.POSITION[ind].zp= pos[2]/10
        
    return cnfA, cnfB

cmol1, cmol2 = align_cnfs_with_MCS(cnfA=cmol1, cnfB=cmol2, atom_mappingAB=atom_mappingAB)



### State merging

In [9]:
cmol2.POSITION

POSITION
# 	 
    1 G078  C12        1   -0.105300000    0.294500000   -0.001000000
    1 G078  C5         2   -0.102000000    0.143400000    0.000100000
    1 G078  C4         3   -0.219200000    0.070800000    0.000700000
    1 G078  H5         4   -0.314600000    0.123200000    0.000500000
    1 G078  C3         5   -0.219200000   -0.070800000    0.000700000
    1 G078  H4         6   -0.314600000   -0.123200000    0.000500000
    1 G078  C2         7   -0.102000000   -0.143400000    0.000100000
    1 G078  C1         8   -0.105300000   -0.294500000   -0.001000000
    1 G078  C11        9    0.022500000   -0.071900000    0.000200000
    1 G078  C6        10    0.022500000    0.071900000    0.000200000
    1 G078  C7        11    0.147500000    0.139700000    0.000200000
    1 G078  H6        12    0.148900000    0.248300000    0.000500000
    1 G078  C8        13    0.266900000    0.070700000   -0.000000000
    1 G078  H7        14    0.361200000    0.124900000    0.000300000
    1 

In [10]:
eucldean_dist = lambda x,y: np.sqrt((x.xp-y.xp)**2+(x.yp-y.yp)**2+(x.zp-y.zp)**2)

def merge_states(cnfA:cnf.Cnf, cnfB:cnf.Cnf, atomMatchingAB:dict, dist_tresh:float=0.09,_doNotChangeAtomType:bool=False, _doUpdateAtomMapping:bool=True)-> cnf.Cnf:#nm
        
    natoms_in_a=len(cnfA.POSITION)
    match_molA, match_molB = np.array(list(atom_mappingAB.items())).T
    atomMatchingAB = copy.deepcopy(atomMatchingAB)
    cmol_comb = copy.deepcopy(cnfA)    
    
    #reduce stateB to required atoms
    for pos in cnfB.POSITION:
        pos2s = [pos2 for pos2 in cnfA.POSITION  if(eucldean_dist(pos, pos2) < dist_tresh)]
        #print(pos.atomID)

        if(len(pos2s) > 0 and _doUpdateAtomMapping):#check distance
            found = False
            for p in pos2s:
                if(not(p.atomID in atomMatchingAB.keys() or pos.atomID in atomMatchingAB.values())):
                    #print("map_Coordinates", pos.atomID, {pos.atomID: p.atomID})
                    atomMatchingAB.update({p.atomID:pos.atomID})
                    found = True
                    break
            if(not (found or pos.atomID in atomMatchingAB.values())):
                #print("adding_Coordinates", pos.atomID)
                pos = copy.deepcopy(pos)
                pos.resID=2
                cmol_comb.POSITION.append(pos)      
            elif(not found):
                #print("already found", pos.atomID, pos.atomID in  atomMatchingAB.values())
                pass
        else:
            #print("adding_Coordinates", pos.atomID)
            pos = copy.deepcopy(pos)
            pos.resID=2
            cmol_comb.POSITION.append(pos)

    #print("LENE: ", len(atomMatchingAB), len(cmol_comb.POSITION))
    #print(cmol_comb.POSITION)
    #print(atomMatchingAB)
    
    #clean cmol_comb
    present_atoms={}
    ati = []
    atomID = 1
    for ind, pos in enumerate(cmol_comb.POSITION):
        #atoms present in coords, mapped to old initial ones
        tID = pos.atomID
        first_state = tID if (pos.resID == 1) else -1
        #print("first", first_state, pos.resID)
        if(first_state != -1):
            if(first_state in atomMatchingAB):
                #print("that if: ", atomMatchingAB[first_state], atomMatchingAB[first_state]+natoms_in_a)
                second_state = atomMatchingAB[first_state]+natoms_in_a
                ati.append(atomMatchingAB[first_state])
            else:
                second_state = -1
        else:
            #print("this if: ", pos.atomID, pos.atomID+natoms_in_a)
            second_state = pos.atomID+natoms_in_a
            ati.append(pos.atomID)

        pos.resID=1
        pos.atomID = atomID
        pos.resName=cmol1.POSITION[0].resName[:2]+cmol2.POSITION[0].resName[2:4]
        if(not _doNotChangeAtomType): pos.atomType = "C"
        present_atoms.update({pos.atomID: (first_state, second_state)})
        atomID+=1        
        
    #print(len(ati), sorted(ati))        
    return cmol_comb, present_atoms

cmol_comb, present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09)
#cmol_comb.write("fung.g96")
#cmol_comb.visualize()
present_atoms

{1: (1, -1),
 2: (2, 16),
 3: (3, 17),
 4: (4, 25),
 5: (5, 26),
 6: (6, 24),
 7: (7, 32),
 8: (8, 22),
 9: (9, 23),
 10: (10, -1),
 11: (11, -1),
 12: (12, 20),
 13: (13, 21),
 14: (14, 18),
 15: (15, 19),
 16: (-1, 27),
 17: (-1, 28),
 18: (-1, 29),
 19: (-1, 30),
 20: (-1, 31),
 21: (-1, 33)}

In [11]:
cmol_comb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Topology

In [12]:
present_atoms

{1: (1, -1),
 2: (2, 16),
 3: (3, 17),
 4: (4, 25),
 5: (5, 26),
 6: (6, 24),
 7: (7, 32),
 8: (8, 22),
 9: (9, 23),
 10: (10, -1),
 11: (11, -1),
 12: (12, 20),
 13: (13, 21),
 14: (14, 18),
 15: (15, 19),
 16: (-1, 27),
 17: (-1, 28),
 18: (-1, 29),
 19: (-1, 30),
 20: (-1, 31),
 21: (-1, 33)}

In [13]:
top1 = top.Top(in_top1_path)
top2 = top.Top(in_top2_path)


In [14]:
com_top = top2
com_top += top1
red_com_top = copy.deepcopy(com_top)

In [15]:
#BONDS
bonds =[]
for bond in com_top.BOND:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
for bond in com_top.BONDH:
    if(bond.IB in present_atoms and bond.JB in present_atoms):
        bonds.append(bond)
red_com_top.BOND.content = bonds
red_com_top.BOND.NBON = len(bonds)

del red_com_top.BONDH


view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, bond in enumerate(red_com_top.BOND):
    view.setStyle({"serial": [bond.IB,  bond.JB]}, {"stick":{'color':colors[i%len(colors)]}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:


#BondAngles
angles =[]
for angle in com_top.BONDANGLE:
    if(angle.IT in present_atoms and angle.JT in present_atoms and angle.KT in present_atoms):
        angles.append(angle)

for angleH in com_top.BONDANGLEH:
    if(angleH.IT in present_atoms and angleH.JT in present_atoms and angleH.KT in present_atoms):
        angles.append(angleH)
        
red_com_top.BONDANGLE.content = angles
red_com_top.BONDANGLE.NTHE = len(angles)

if(hasattr(red_com_top, "BONDANGLEH")): del red_com_top.BONDANGLEH


view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, angle in enumerate(red_com_top.BONDANGLE):
    view.setStyle({"serial": [angle.IT,  angle.JT, angle.KT]}, {"stick":{'color':colors[i%len(colors)]}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
com_top.DIHEDRALH

DIHEDRALH
# 	 NPHIH: number of torsion dihedrals involving H atoms
 	 2
# 	 IPH 	 JPH 	 KPH 	 LPH 	 ICPH
	24	26	27	29	10
	30	26	27	28	10
END

In [18]:
red_com_top.DIHEDRAL

DIHEDRAL
# 	 NPHI: number of dihedrals NOT involving H atoms
 	 18
# 	 IP 	 JP 	 KP 	 LP 	 ICP
	10	2	3	5	15
	3	2	10	9	15
	2	3	5	7	15
	3	5	7	9	15
	5	7	9	10	15
	17	9	10	11	15
	10	9	17	15	15
	9	10	11	13	15
	10	11	13	15	15
	11	13	15	17	15
	13	15	17	9	15
	19	20	21	22	13
	32	21	22	24	15
	22	21	32	30	15
	21	22	24	26	15
	22	24	26	30	15
	24	26	30	32	15
	26	30	32	21	15
END

In [19]:
present_atoms

{1: (1, -1),
 2: (2, 16),
 3: (3, 17),
 4: (4, 25),
 5: (5, 26),
 6: (6, 24),
 7: (7, 32),
 8: (8, 22),
 9: (9, 23),
 10: (10, -1),
 11: (11, -1),
 12: (12, 20),
 13: (13, 21),
 14: (14, 18),
 15: (15, 19),
 16: (-1, 27),
 17: (-1, 28),
 18: (-1, 29),
 19: (-1, 30),
 20: (-1, 31),
 21: (-1, 33)}

In [20]:
#Dihedrals
dihedrals =[]
for dih in com_top.DIHEDRAL:
    if(dih.IP in present_atoms and dih.JP in present_atoms and dih.KP in present_atoms and dih.LP in present_atoms):
        dihedrals.append(dih)

dih_type = type(dih)
for dihh in com_top.DIHEDRALH:
    if(dihh.IPH in present_atoms and dihh.JPH in present_atoms and dihh.KPH in present_atoms and dihh.LPH in present_atoms):
        dihedrals.append(dih_type(IP=dihh.IPH, JP=dihh.JPH, KP=dihh.KPH, LP=dihh.LPH, ICP=dihh.ICPH))
        
red_com_top.DIHEDRAL.content = dihedrals
red_com_top.DIHEDRAL.NPHI = len(dihedrals)

if(hasattr(red_com_top, "DIHEDRALH")): del red_com_top.DIHEDRALH

    
view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, tors in enumerate(red_com_top.DIHEDRAL):
    view.setStyle({"serial": [tors.IP,  tors.JP, tors.KP, tors.LP]}, {"stick":{'color':colors[i%len(colors)]}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [21]:



#Impropers
impdihedrals =[]
for imp in com_top.IMPDIHEDRAL:
    if(imp.IQ in present_atoms and imp.JQ in present_atoms and imp.KQ in present_atoms and imp.LQ in present_atoms):
        impdihedrals.append(imp)
        
imp_type = type(imp)
for impH in com_top.IMPDIHEDRALH:
    if(impH.IQH in present_atoms and impH.JQH in present_atoms and impH.KQH in present_atoms and impH.LQH in present_atoms):
        impdihedrals.append(imp_type(IQ=impH.IQH, JQ=impH.JQH, KQ=impH.KQH, LQ=impH.LQH, ICQ=impH.ICQH))
        
red_com_top.IMPDIHEDRAL.content = impdihedrals
red_com_top.IMPDIHEDRAL.NQHI = len(impdihedrals)

if(hasattr(red_com_top, "IMPDIHEDRALH")): del red_com_top.IMPDIHEDRALH

view = cmol_comb.visualize()

colors = ['green', 'blue', 'yellow', 'red', 'purple', 'black']
for i, tors in enumerate(red_com_top.IMPDIHEDRAL):
    view.setStyle({"serial": [tors.IQ,  tors.JQ, tors.KQ, tors.LQ]}, {"stick":{'color':colors[i%len(colors)]}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
vars(tors)

{'IQ': 17, 'JQ': 9, 'KQ': 15, 'LQ': 18, 'ICQ': 1}

In [23]:

#Mass, VDW, Charge
sol =[]
for atm in com_top.SOLUTEATOM:
    if(atm.ATNM in present_atoms):
        atm.MRES=1
        sol.append(atm)

red_com_top.SOLUTEATOM.content = sol 
red_com_top.SOLUTEATOM.NRP = len(sol)

"""
#crossDIHEDRAL
crds =[]
for crd in com_top.CROSSDIHEDRAL:
    print(vars(crd))
    if(crd.ATNM in present_atoms):
        crds.append(atm)

red_com_top.CROSSDIHEDRAL.content = crds 

#crossDIHEDRALH
crdsH =[]
for crdH in com_top.CROSSDIHEDRALH:
    print(vars(crdH))
    if(crdH.ATNM in present_atoms):
        crdsH.append(atm)

red_com_top.CROSSDIHEDRALH.content = crdsH
"""


'\n#crossDIHEDRAL\ncrds =[]\nfor crd in com_top.CROSSDIHEDRAL:\n    print(vars(crd))\n    if(crd.ATNM in present_atoms):\n        crds.append(atm)\n\nred_com_top.CROSSDIHEDRAL.content = crds \n\n#crossDIHEDRALH\ncrdsH =[]\nfor crdH in com_top.CROSSDIHEDRALH:\n    print(vars(crdH))\n    if(crdH.ATNM in present_atoms):\n        crdsH.append(atm)\n\nred_com_top.CROSSDIHEDRALH.content = crdsH\n'

In [24]:
#REST
##resname meshing
molA_Name = com_top.RESNAME.content[1][0]
molB_Name = com_top.RESNAME.content[2][0]

red_com_top.RESNAME.content = [['1'], [molA_Name[:2]+molB_Name[:2]]]


## SOLUTEMOLECULES:
from collections import defaultdict
resi_atom = defaultdict(int)

for atom in red_com_top.SOLUTEATOM:
    resi_atom[atom.MRES]+=1

groups = [[str(len(resi_atom))], [str(resi_atom[res])+" " for res in resi_atom]]
red_com_top.SOLUTEMOLECULES.content =groups
red_com_top.TEMPERATUREGROUPS.content =groups
red_com_top.PRESSUREGROUPS.content =groups

pass

### Build PTP

In [25]:
from pygromos.files.blocks.pertubation_blocks import TITLE, PERTBONDSTRETCH, PERTBONDSTRETCHH, atom_lam_pertubation_state_bond
from pygromos.files.blocks.topology_blocks import bondstretchtype_type, torsdihedraltype_type, impdihedraltype_type

### DummyTypes

In [26]:
#Dummy Types
dummyBond = bondstretchtype_type(CB=0, CHB=0, B0=1)
dummyAngle = bondstretchtype_type(CB=0, CHB=0, B0=1)
dummyTors = torsdihedraltype_type(CP=0, PD=0, NP=1)
dummyImp = impdihedraltype_type(CQ=0, Q0=0)

dummyIAC = 21
dummyMass = -1
dummyCharge = 0.0

#Add to top
red_com_top.BONDSTRETCHTYPE.append(dummyBond)
red_com_top.BONDANGLEBENDTYPE.append(dummyAngle)
red_com_top.TORSDIHEDRALTYPE.append(dummyTors)
red_com_top.IMPDIHEDRALTYPE.append(dummyImp)


dummyBondInd = len(red_com_top.BONDSTRETCHTYPE)
dummyAngleInd = len(red_com_top.BONDANGLEBENDTYPE)
dummyTorsInd = len(red_com_top.TORSDIHEDRALTYPE)
dummyIMPsInd = len(red_com_top.IMPDIHEDRALTYPE)

red_com_top.BONDSTRETCHTYPE.NBTY = dummyBondInd
red_com_top.BONDANGLEBENDTYPE.NBTY = dummyAngleInd
red_com_top.TORSDIHEDRALTYPE.NPTY = dummyTorsInd
red_com_top.IMPDIHEDRALTYPE.NQTY = dummyIMPsInd


In [78]:
def reduce_pertubations(pertubations):
    #reduce
    atom_keys = [k for k in vars(pertubations[0]) if("atom" in k)]
    stateAtoms = list(sorted(pertubations, key=lambda x: [vars(x)[k] for k in atom_keys]))
    skip = []
    reduced_pertubations = []
    for i,bond1 in enumerate(stateAtoms):
        if(bond1 in skip):
            continue
        else:
            #is there already a bond?
            attr = vars(bond1)
            for bond2  in stateAtoms[i+1:]:
                if(set([attr[k] for k in atom_keys]) == set([vars(bond2)[k] for k in atom_keys])):
                    skip.append(bond2)
                    replace_keys = [k for k in attr['STATES'] if (attr['STATES'][k] == dummyBondInd)]
                    if(len(replace_keys)>0):
                        replace_key = replace_keys[0]
                        attr['STATES'].update({replace_key: bond2.STATES[replace_key]})
                        break
                    else:
                        continue

            #something equal already there?
            if(attr['STATES'][0] == attr['STATES'][1] or 
               set([attr[k] for k in atom_keys]) in [set([vars(x)[k] for k in atom_keys]) for x in reduced_pertubations]):
                continue
            else:
                attr.update({"NR":len(reduced_stateAtoms)})
                new_pertAtomState = type(bond1)(**attr)
                reduced_pertubations.append(new_pertAtomState)
    return reduced_pertubations


def merge_tops_to_pertubations(tops, pertType, topBlockName, dummystateInd,  ):
    pertubations = []
    tops = [top1, top2]
    toffset = 0
    for tind, ttop in enumerate(tops):
        rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
        for top_bond in getattribute(ttop, topBlockName):
            
            tstates = {tind:top_bond.ICB}
            tstates.update({i:dummystateInd for i in range(len(tops)) if not i in tstates})
            pertubations.append(pertType(NR=len(pertBonds), 
                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                         STATES=tstates)
                                )
        toffset = len(ttop.SOLUTEATOM)    
    
    return pertubations

### Bond

In [57]:
pertbond =PERTBONDSTRETCH()

pertBonds = []
tops = [top1, top2]
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_bond in ttop.BOND:
        tstates = {tind:top_bond.ICB}
        tstates.update({i:dummyBondInd for i in range(len(tops)) if not i in tstates})
        pertBonds.append(atom_lam_pertubation_state_bond(NR=len(pertBonds), 
                                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_bond in ttop.BONDH:
        tstates = {tind:top_bond.ICB}
        tstates.update({i:dummyBondInd for i in range(len(tops)) if not i in tstates})
        pertBonds.append(atom_lam_pertubation_state_bond(NR=len(pertBonds), 
                                                         atomI=rearrange_dict[top_bond.IB+toffset],
                                                         atomJ=rearrange_dict[top_bond.JB+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           



#reduce
reduced_stateAtoms = reduce_pertubations(stateAtoms)
        
pertbond.STATEATOMS = reduced_stateAtoms
pertbond.NPB = len(reduced_stateAtoms)

pertbond

PERTBONDSTRETCH
# NPB  	 NPTB =  	 2 	 
 	 19
# atomI atomJ type1  type2 
    1     2     3    53   156
    2     3     4    22   156
    2     3    14    22   156
    3     4     5   156    27
    3     4     6    27   156
    3    14    12    27   156
    4     3    14    27   156
    4     5    17   156    27
    4     6     7   156    27
    4     6     8    27   156
    5    17    19   156    27
    6     4     5   156    27
    6     7    19   156    27
    6     8     9    27   156
    6     8    12    27   156
    8     6     7   156    27
    8    12    14    27   156
    9     8    12    27   156
   17    19     7   156    27
END

### Angle

In [28]:
from pygromos.files.blocks.pertubation_blocks import PERTBONDANGLE, PERTBONDANGLEH, atom_lam_pertubation_state_angle

In [58]:
pertangle =PERTBONDANGLE()

pertAngles = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_angle in ttop.BONDANGLE:
        tstates = {tind:top_angle.ICT}
        tstates.update({i:dummyAngleInd for i in range(len(tops)) if not i in tstates})
        pertAngles.append(atom_lam_pertubation_state_angle(NR=len(pertAngles), 
                                                         atomI=rearrange_dict[top_angle.IT+toffset],
                                                         atomJ=rearrange_dict[top_angle.JT+toffset], 
                                                         atomK=rearrange_dict[top_angle.KT+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           
    
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_angle in ttop.BONDANGLE:
        tstates = {tind:top_angle.ICT}
        tstates.update({i:dummyAngleInd for i in range(len(tops)) if not i in tstates})
        pertAngles.append(atom_lam_pertubation_state_angle(NR=len(pertAngles), 
                                                         atomI=rearrange_dict[top_angle.IT+toffset],
                                                         atomJ=rearrange_dict[top_angle.JT+toffset], 
                                                         atomK=rearrange_dict[top_angle.KT+toffset], 
                                                         STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)    
    
    

#reduce
reduced_pertAngles = reduce_pertubations(pertAngles)
            
pertangle.STATEATOMS = reduced_pertAngles
pertangle.NPA = len(reduced_pertAngles)

pertangle

PERTBONDANGLE
# NPA  	 NPTB =  	 2 	 
 	 19
# atomI atomJ atomK type1  type2 
    1     2     3    53   156
    2     3     4    22   156
    2     3    14    22   156
    3     4     5   156    27
    3     4     6    27   156
    3    14    12    27   156
    4     3    14    27   156
    4     5    17   156    27
    4     6     7   156    27
    4     6     8    27   156
    5    17    19   156    27
    6     4     5   156    27
    6     7    19   156    27
    6     8     9    27   156
    6     8    12    27   156
    8     6     7   156    27
    8    12    14    27   156
    9     8    12    27   156
   17    19     7   156    27
END

### Dihedrals

In [54]:
from pygromos.files.blocks.pertubation_blocks import PERTPROPERDIH, PERTPROPERDIHH, atom_lam_pertubation_state_dihedral

In [60]:
pertproperdih =PERTPROPERDIH()

pertDihedrals = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_dihedral in ttop.DIHEDRAL:
        tstates = {tind:top_dihedral.ICP}
        tstates.update({i:dummyTorsInd for i in range(len(tops)) if not i in tstates})
        pertDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertDihedrals), 
                                                             atomI=rearrange_dict[top_dihedral.IP+toffset],
                                                             atomJ=rearrange_dict[top_dihedral.JP+toffset], 
                                                             atomK=rearrange_dict[top_dihedral.KP+toffset],
                                                             atomL=rearrange_dict[top_dihedral.LP+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)   
    
toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_dihedral in ttop.DIHEDRALH:
        tstates = {tind:top_dihedral.ICPH}
        tstates.update({i:dummyTorsInd for i in range(len(tops)) if not i in tstates})
        pertDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertDihedrals), 
                                                             atomI=rearrange_dict[top_dihedral.IPH+toffset],
                                                             atomJ=rearrange_dict[top_dihedral.JPH+toffset], 
                                                             atomK=rearrange_dict[top_dihedral.KPH+toffset],
                                                             atomL=rearrange_dict[top_dihedral.LPH+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)  

#reduce
reduced_pertDihedrals = reduce_pertubations(pertDihedrals)

    
pertproperdih.STATEATOMS = reduced_pertDihedrals
pertproperdih.NPD = len(reduced_pertDihedrals)

pertproperdih

PERTPROPERDIH
# NPD  	 NPTB =  	 2 	 
 	 15
# atomI atomJ atomK atomL type1  type2 
    1     2     3     4    13    47
    3     4     6     8    15    47
    3    14    12     8    47    15
    4     3    14    12    15    47
    4     5    17    19    47    15
    4     6     7    19    47    15
    4     6     8    12    15    47
    5    17    19     7    47    15
    6     4     5    17    47    15
    6     8     9    11    10    47
    6     8    12    14    15    47
    7     6     4     5    47    15
   12     8     9    10    10    47
   14     3     4     6    15    47
   17    19     7     6    47    15
END

### Improper Dihedrals

In [32]:
from pygromos.files.blocks.pertubation_blocks import PERTIMROPERDIH, PERTIMROPERDIHH, atom_lam_pertubation_state_dihedral

In [62]:
pertproperimpdih =PERTIMROPERDIH()

pertImpDihedrals = []
tops = [top1, top2]

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_imp in ttop.IMPDIHEDRAL:
        tstates = {tind:top_imp.ICQ}
        tstates.update({i:dummyIMPsInd for i in range(len(tops)) if not i in tstates})
        pertImpDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertImpDihedrals), 
                                                             atomI=rearrange_dict[top_imp.IQ+toffset],
                                                             atomJ=rearrange_dict[top_imp.JQ+toffset], 
                                                             atomK=rearrange_dict[top_imp.KQ+toffset],
                                                             atomL=rearrange_dict[top_imp.LQ+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)           

toffset = 0
for tind, ttop in enumerate(tops):
    rearrange_dict = {val[tind]: key for key,val in present_atoms.items()}
    for top_impH in ttop.IMPDIHEDRALH:
        tstates = {tind:top_impH.ICQH}
        tstates.update({i:dummyIMPsInd for i in range(len(tops)) if not i in tstates})
        pertImpDihedrals.append(atom_lam_pertubation_state_dihedral(NR=len(pertImpDihedrals), 
                                                             atomI=rearrange_dict[top_impH.IQH+toffset],
                                                             atomJ=rearrange_dict[top_impH.JQH+toffset], 
                                                             atomK=rearrange_dict[top_impH.KQH+toffset],
                                                             atomL=rearrange_dict[top_impH.LQH+toffset], 
                                                             STATES=tstates)
                            )
    toffset = len(ttop.SOLUTEATOM)        

reduced_pertImpDihedrals = reduce_pertubations(pertImpDihedrals)

pertproperimpdih.STATEATOMS = reduced_pertImpDihedrals
pertproperimpdih.NPD = len(reduced_pertImpDihedrals)

pertproperimpdih

PERTIMROPERDIH
# NPD  	 NPTB =  	 2 	 
 	 10
# atomI atomJ atomK atomL type1  type2 
    3     2     4    14     1     6
    4     3     5     6     1     6
    5     4    16    17     6     1
    6     4     7     8     1     6
    7     6    19    21     6     1
    8     6     9    12     1     6
   12     8    13    14     1     6
   14     3    12    15     1     6
   17     5    18    19     6     1
   19    17    20     7     6     1
END


### NONBONDEDS

In [63]:
from pygromos.files.blocks.pertubation_blocks import PERTATOMPARAM, atom_lam_pertubation_state, pertubation_lam_state_nonbonded

In [64]:
nonbonded = PERTATOMPARAM()

nonbondeds = []
tops = [top1, top2]
toffset = 0

dummyState =lambda m: pertubation_lam_state_nonbonded(IAC=dummyIAC, MASS=m, CHARGE=dummyCharge)

for atom in red_com_top.SOLUTEATOM:
    atom_states = present_atoms[atom.ATNM]
    tstates = {}
    for state, atomNR in enumerate(atom_states):
        if(atomNR == -1):
            tstates.update({state :dummyState(atom.MASS)})
        else:
            toffset = sum([len(tops[state].SOLUTEATOM) for i in range(state)])
            atom_nb = tops[state].SOLUTEATOM[atomNR-toffset-1]
            tstates.update({state:pertubation_lam_state_nonbonded(IAC=atom_nb.IAC, MASS=atom_nb.MASS, CHARGE=atom_nb.CG)})
                  

    nonbondeds.append(atom_lam_pertubation_state(NR=atom.ATNM,
                                                 RES=1,
                                                 NAME=atom_nb.PANM,
                                                 STATES=tstates)
                        )    
nonbonded.STATEATOMS = nonbondeds
nonbonded.NJLA = len(nonbondeds)
nonbonded.STATEIDENTIFIERS = ["state"+str(i) for i in range(len(tops))]


nonbonded

PERTATOMPARAM
# NJLA  	 NPTB =  	 2 	 
 	 21
# state_identifiers
#  	 state0 	 state1
#    NR   RES  NAME   IAC MASS    CHARGE   IAC MASS    CHARGE    ALPHLJ     ALPHCRF   
    1     1    C5    16 15.035    0.27500    21 15.035    0.00000    1.00000    1.00000
    2     1    H8     4 15.9994   -0.40000    20 1.008    0.13100    1.00000    1.00000
    3     1   C10    64 12.011    0.26900    64 12.011   -0.20100    1.00000    1.00000
    4     1    C2    64 12.011   -0.21400    64 12.011   -0.03200    1.00000    1.00000
    5     1    C1    20 1.008    0.14400    16 15.035    0.07200    1.00000    1.00000
    6     1    H4    64 12.011   -0.33400    20 1.008    0.15900    1.00000    1.00000
    7     1    H7    20 1.008    0.18600    20 1.008    0.13100    1.00000    1.00000
    8     1    H5    64 12.011    0.46200    20 1.008    0.15900    1.00000    1.00000
    9     1    C3    66 14.0067   -0.95000    64 12.011   -0.22200    1.00000    1.00000
   10     1    H1    59 1.008    0.3900

### BUild File

In [65]:
red_com_top.RESNAME

RESNAME
1
G0F3
END

In [66]:
ptp_file = ptp.Ptp()


In [67]:
ptp_file.add_block(block=TITLE("\tpertubations for "+"\n ".join(map(lambda x: "\t".join(x), red_com_top.RESNAME.content))))
ptp_file.add_block(block=pertbond)
ptp_file.add_block(block=pertangle)
ptp_file.add_block(block=pertproperdih)
ptp_file.add_block(block=pertproperimpdih)

ptp_file.add_block(block=nonbonded)

In [68]:
ptp_file

TITLE
	pertubations for 1 G0F3
	 >>> Generated with python lib function_libs utilities. (riniker group)
	 >>> line_seperator: '\n'	 field_seperator: '\t'	 comments_char: '#'
END
PERTBONDSTRETCH
# NPB  	 NPTB =  	 2 	 
 	 19
# atomI atomJ type1  type2 
    1     2     3    53   156
    2     3     4    22   156
    2     3    14    22   156
    3     4     5   156    27
    3     4     6    27   156
    3    14    12    27   156
    4     3    14    27   156
    4     5    17   156    27
    4     6     7   156    27
    4     6     8    27   156
    5    17    19   156    27
    6     4     5   156    27
    6     7    19   156    27
    6     8     9    27   156
    6     8    12    27   156
    8     6     7   156    27
    8    12    14    27   156
    9     8    12    27   156
   17    19     7   156    27
END
PERTBONDANGLE
# NPA  	 NPTB =  	 2 	 
 	 19
# atomI atomJ atomK type1  type2 
    1     2     3    53   156
    2     3     4    22   156
    2     3    14    22   156
    3 

# Check Pertubation top

In [69]:
#reduce
stateAtoms = list(sorted(ptp_file.PERTBONDSTRETCH.STATEATOMS, key=lambda x: (x.atomI, x.atomJ)))
skip = []
reduced_stateAtoms = []
for i,bond1 in enumerate(stateAtoms):
    if(bond1 in skip):
        print('skip', bond1)
        continue
    else:
        #is there already a bond?
        attr = vars(bond1)
        for bond2  in stateAtoms[i+1:]:
            if(all([vars(bond1)[k]==vars(bond2)[k] for k in vars(bond1) if("atom" in k)])):
                print(bond1, bond2)
                skip.append(bond2)
                
                replace_keys = [k for k in attr['STATES'] if (attr['STATES'][k] == dummyBondInd)]
                if(len(replace_keys)>0):
                    replace_key = replace_keys[0]
                    print(attr, replace_key)

                    attr['STATES'].update({replace_key: bond2.STATES[replace_key]})
                    print(attr)
                    break
                else:
                    continue
        print(attr)
        if(attr['STATES'][0] == attr['STATES'][1]):
            continue
        attr.update({"NR":len(reduced_stateAtoms)})
        new_pertAtomState = type(bond1)(**attr)
        reduced_stateAtoms.append(new_pertAtomState)

ptp_file.PERTBONDSTRETCH.STATEATOMS =  reduced_stateAtoms             
ptp_file.PERTBONDSTRETCH.NPB = len(reduced_stateAtoms)
ptp_file.PERTBONDSTRETCH

{'NR': 20, 'atomI': 1, 'atomJ': 2, 'atomK': 3, 'STATES': {0: 53, 1: 156}}
{'NR': 20, 'atomI': 2, 'atomJ': 3, 'atomK': 4, 'STATES': {0: 22, 1: 156}}
{'NR': 20, 'atomI': 2, 'atomJ': 3, 'atomK': 14, 'STATES': {0: 22, 1: 156}}
{'NR': 20, 'atomI': 3, 'atomJ': 4, 'atomK': 5, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 3, 'atomJ': 4, 'atomK': 6, 'STATES': {0: 27, 1: 156}}
{'NR': 20, 'atomI': 3, 'atomJ': 14, 'atomK': 12, 'STATES': {0: 27, 1: 156}}
{'NR': 20, 'atomI': 4, 'atomJ': 3, 'atomK': 14, 'STATES': {0: 27, 1: 156}}
{'NR': 20, 'atomI': 4, 'atomJ': 5, 'atomK': 17, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 4, 'atomJ': 6, 'atomK': 7, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 4, 'atomJ': 6, 'atomK': 8, 'STATES': {0: 27, 1: 156}}
{'NR': 20, 'atomI': 5, 'atomJ': 17, 'atomK': 19, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 6, 'atomJ': 4, 'atomK': 5, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 6, 'atomJ': 7, 'atomK': 19, 'STATES': {1: 27, 0: 156}}
{'NR': 20, 'atomI': 6, 'atomJ'

PERTBONDSTRETCH
# NPB  	 NPTB =  	 2 	 
 	 19
# atomI atomJ type1  type2 
    1     2     3    53   156
    2     3     4    22   156
    2     3    14    22   156
    3     4     5   156    27
    3     4     6    27   156
    3    14    12    27   156
    4     3    14    27   156
    4     5    17   156    27
    4     6     7   156    27
    4     6     8    27   156
    5    17    19   156    27
    6     4     5   156    27
    6     7    19   156    27
    6     8     9    27   156
    6     8    12    27   156
    8     6     7   156    27
    8    12    14    27   156
    9     8    12    27   156
   17    19     7   156    27
END

In [71]:
#write out all files
ptp_file_path = ptp_file.write(out_ptp_path)
top_file_path = red_com_top.write(out_top_path)
cnf_file_path = cmol_comb.write(out_cnf_path)

## Test Emin:


In [72]:
from pygromos.files.gromos_system import Gromos_System
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.simulations.modules.preset_simulation_modules import emin
from pygromos.hpc_queuing.submission_systems.local import LOCAL as subSystem


In [73]:
root_dir = "/home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop"
template_emin

'/home/bschroed/Documents/projects/PyGromosTools/pygromos/data/simulation_parameters_templates/emin.imd'

In [74]:
sys_name = 'test'
grom_system = Gromos_System(in_cnf_path=cnf_file_path, in_top_path=top_file_path, in_imd_path=template_emin,
                            in_ptp_path=ptp_file_path,
                            system_name=sys_name, 
                            work_folder=root_dir+"/out")



	CHECK dependencies

Check /home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop/out
Check /home/bschroed/Documents/projects/PyGromosTools/pygromos/data/simulation_parameters_templates/emin.imd
Check ./out/fun.top
Check ./out/fun.ptp
Check ./out/coord.g96
/home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop/out
/home/bschroed/Documents/projects/PyGromosTools/pygromos/data/simulation_parameters_templates/emin.imd
./out/fun.top
./out/fun.ptp
./out/coord.g96
All dependencies are correct! 


Parsing File:  imd
Parsing File:  top
Parsing File:  ptp
Parsing File:  cnf


/home/bschroed/Documents/projects/PyGromosTools/pygromos/files/gromos_system/gromos_system.py:131: UserWarning: No data provided to gromos_system
manual work needed
  if verbose: warnings.warn("No data provided to gromos_system\nmanual work needed")


In [75]:
from pygromos.files.blocks.imd_blocks import PERTURBATION
lam =1
pert_block  =  PERTURBATION(NTG=1, NRDGL=0, RLAM=lam, DLAMT=0,
                            ALPHC=0.5, ALPHLJ=0.5, NLAM=2, NSCALE=0)
grom_system.imd.add_block(block=pert_block)

In [76]:
grom_system.imd.INITIALISE.NTISHK = 0

help(grom_system.imd.INITIALISE)

Help on INITIALISE in module pygromos.files.blocks.imd_blocks object:

class INITIALISE(_generic_imd_block)
 |  INITIALISE(NTIVEL: bool = False, NTISHK: int = 0, NTINHT: bool = False, NTINHB: bool = False, NTISHI: bool = False, NTIRTC: bool = False, NTICOM: int = 0, NTISTI: bool = False, IG: int = 0, TEMPI: float = 0, content=None)
 |  
 |  INITIALISE block
 |  
 |      This block controls the Initialisation of a simulation.
 |  
 |  Attributes
 |  ----------
 |  NTIVEL: bool
 |      0,1 controls generation of initial velocities
 |      0: read from configuration (default)
 |      1: generate from Maxell distribution at temperature TEMPI
 |  NTISHK: int
 |      0..3 controls shaking of initial configuration
 |      0: no intial SHAKE (default)
 |      1: initial SHAKE on coordinates only
 |      2: initial SHAKE on velocities only
 |      3: initial SHAKE on coordinates and velocities
 |  NTINHT: bool
 |      0,1 controls generation of initial Nose-Hoover chain variables
 |      0: rea

In [ ]:
import os

subSys = subSystem(verbose=True)
emin_gromos_system, jobID = emin(in_gromos_system=grom_system, project_dir=root_dir, 
                                 submission_system=subSys, in_imd_path=None)



	CHECK dependencies

Check /home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop/out
/home/bschroed/Documents/projects/PyGromosTools/examples/dev/singleTop/out
All dependencies are correct! 


Generate Empty:  imd
Generate Empty:  top
Generate Empty:  cnf
################################################################################

emin
################################################################################

Script:  /home/bschroed/Documents/projects/PyGromosTools/pygromos/hpc_queuing/job_scheduling/schedulers/simulation_scheduler.py
################################################################################
Simulation PREPERATION
################################################################################



################################################################################
Simulation Setup:
################################################################################

steps_per_run:  3000
equis:  0
simulation runs:  1
Searchin

In [ ]:
emin_cnf = cnf.Cnf(root_dir+"/emin/simulation/emin_1/emin_1.cnf")

In [ ]:
emin_cnf.visualize()

## DONE

#### Dual Topology Approach

In [ ]:
def generate_dual_topology_approach(cnfA, cnfB, topA, topB, eds:bool=False):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb = copy.deepcopy(cnfA)
    #cnf_comb += cnfB # needs to be implemented
    
    for pos in cnfB.POSITION:
        cnf_comb.POSITION.append(pos)
    
    cnf_comb.supress_atomPosition_singulrarities()


    ##Top
    nAtoms_top1 = len(top1.SOLUTEATOM)
    top_comb = copy.deepcopy(top1)
    top_comb += top2
    
    ### Pertubation
    ptp_comb = ptp.Pertubation_topology()
    
    if(eds):
        from pygromos.files.blocks.pertubation_blocks import MPERTATOM
        from pygromos.files.blocks.pertubation_blocks import  atom_eds_pertubation_state, pertubation_eds_state

        tops = [topA, topB]
        dummyState = pertubation_eds_state(IAC=22, CHARGE=0)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_eds_state(IAC=atom.IAC, CHARGE=atom.CG)})
                    else:
                        states.update({ctop+1:dummyState})

                atom_ptp = atom_eds_pertubation_state(NR=IND, NAME=atom.PANM, STATES=states)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=MPERTATOM(NJLA=len(atom_states), NPTB=numStates, STATEATOMS=atom_states))

    else:
        from pygromos.files.blocks.pertubation_blocks import PERTATOMPARAM
        from pygromos.files.blocks.pertubation_blocks import  atom_lam_pertubation_state, pertubation_lam_state_nonbonded

        tops = [topA, topB]
        build_dummyState = lambda m: pertubation_lam_state_nonbonded(IAC=22, CHARGE=0, MASS=m)

        numStates=len(tops)
        IND = 1
        atom_states = []
        for top_ind, top in enumerate(tops):
            for atom in top.SOLUTEATOM:        
                states = {}
                for ctop in range(ntops):
                    if(ctop==top_ind):
                        states.update({ctop+1:pertubation_lam_state_nonbonded(IAC=atom.IAC, CHARGE=atom.CG, MASS=atom.MASS)})
                    else:
                        states.update({ctop+1:build_dummyState(atom.MASS)})

                atom_ptp = atom_lam_pertubation_state(NR=IND, RES=atom.MRES, NAME=atom.PANM, STATES=states,)
                atom_states.append(atom_ptp)
                IND+=1

        ptp_comb.add_block(block=PERTATOMPARAM(NJLA=len(atom_states), STATEATOMS=atom_states))
    
    return cnf_comb, top_comb, ptp_comb

In [ ]:
cnf_file, top_file, ptp_file = generate_dual_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
top_file.SOLUTEATOM

In [ ]:
ptp_file

#### Hybrid Topology Approach

In [ ]:
def generate_hybrid_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb, present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.0, _doNotChangeAtomType=True, _doUpdateAtomMapping=True) #no distance collapsing
    
    ##Top
    
    
    ### Pertubation
    
    
    return cnf_comb
    #return cnf_comb, top_comb, ptp_comb

In [ ]:
cnf_file= generate_hybrid_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
cnf_file

#### Single Topology Approach

In [ ]:
def generate_single_topology_approach(cnfA, cnfB, topA, topB):
    ##Atom Mapping
    atom_mappingAB, smart = find_atom_mapping(cnfA=cnfA, cnfB=cnfB)
    
    ##Coordinates
    cnfA, cnfB = align_cnfs_with_MCS(cnfA=cnfA, cnfB=cnfB, atom_mappingAB=atom_mappingAB)
    cnf_comb,present_atoms = merge_states(cnfA=cmol1, cnfB=cmol2, atomMatchingAB=atom_mappingAB, dist_tresh=0.09) #no distance collapsing
    

    return cnf_comb #, top_comb, ptp_comb

In [ ]:
cnf_file= generate_single_topology_approach(cnfA=cmol1, cnfB=cmol2, topA=top1, topB=top2)

In [ ]:
cnf_file.visualize()

In [ ]:
cnf_file